In [23]:
!pip install chatterbot chatterbot_corpus
!pip install fuzzywuzzy
!pip install python-Levenshtein
!pip install pytz
!python -m spacy download en_core_web_sm
!pip install nltk



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\lapto\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\lapto\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\lapto\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\lapto\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 245.8 kB/s eta 0:00:52
     --------------------------------------- 0.1/12.8 MB 393.8 kB/s eta 0:00:33
     --------------------------------------- 0.1/12.8 MB 554.9 kB/s eta 0:00:23
      -------------------------------------- 0.2/12.8 MB 737.3 kB/s eta 0:00:18
      -------------------------------------- 0.2/12.8 MB 801.7 kB/s eta 0:00:16
      -------------------------------------- 0.3/12.8 MB 863.3 kB/s eta 0:00:15
     - ------------------------------------- 0.4/12.8 MB 916.6 kB/s eta 0:00:14
     - ------------------------------------- 0.4/12.8 MB 935.2 kB/s eta 0:00:14
     - -------------------------------------- 0.5/12.8 MB 1.0 MB/s eta 0:00:13
     - -------------------------------------- 0.6/12.8 MB 1.1 MB/


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\lapto\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\lapto\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [24]:
import pytz
import random
import re
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer
from fuzzywuzzy import fuzz
from datetime import datetime
import nltk
from nltk.corpus import wordnet


In [25]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\lapto\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
# Create a chatbot instance with SQL storage
chatbot = ChatBot(
    "MyBot",
    storage_adapter="chatterbot.storage.SQLStorageAdapter",
    database_uri="sqlite:///chatbot_db.sqlite3",
    logic_adapters=[
        "chatterbot.logic.BestMatch",  # Best matching response
        "chatterbot.logic.MathematicalEvaluation",  # Handles math questions
        "chatterbot.logic.TimeLogicAdapter"  # Tells the current time
    ]
)

In [27]:
trainer = ChatterBotCorpusTrainer(chatbot)
trainer.train("chatterbot.corpus.english")

ChatterBot Corpus Trainer: 19it [00:03,  5.82it/s]


In [28]:
custom_trainer = ListTrainer(chatbot)
custom_conversations = [
    ["Hello", "Hi there! How can I help you?"],
    ["How are you?", "I'm just a BOT, but I'm doing well!"],
    ["What is your name?", "I am FRIDAY, your AI assistant."],
    ["Bye", "Goodbye! Have a great day!"]
]

for conv in custom_conversations:
    custom_trainer.train(conv)

List Trainer: 2it [00:00, 500.27it/s]
List Trainer: 2it [00:00, 500.78it/s]
List Trainer: 2it [00:00, 500.27it/s]
List Trainer: 2it [00:00, 664.92it/s]


In [29]:
jokes = [
    "Tell me a joke",
    "Why don’t skeletons fight each other? They don’t have the guts!",
    "Why don’t scientists trust atoms? Because they make up everything!",
    "Why did the scarecrow win an award? Because he was outstanding in his field!",
    "Why did the bicycle fall over? Because it was two-tired!",
    "What do you call cheese that isn't yours? Nacho cheese!"
]
custom_trainer.train(jokes)

List Trainer: 6it [00:00, 683.13it/s]


In [30]:
math_examples = [
    "What is 5 + 3?",
    "The answer is 8.",
    "What is 10 divided by 2?",
    "That would be 5."
]
custom_trainer.train(math_examples)

List Trainer: 4it [00:00, 1000.73it/s]


In [31]:
def is_repeat_request(user_input):
    repeat_keywords = ["again", "another one", "one more", "repeat", "give me another", "tell me again"]
    
    for phrase in repeat_keywords:
        if fuzz.ratio(user_input.lower(), phrase) > 70:  # 70% similarity threshold
            return True
    return False

In [32]:
def is_math_expression(user_input):
    return bool(re.match(r"^\s*-?\d+(\s*[\+\-\*/]\s*-?\d+)+\s*$", user_input))


In [33]:
def get_current_time():
    ist = pytz.timezone('Asia/Kolkata')  # IST timezone
    current_time = datetime.now(ist).strftime("%I:%M %p")  # Format: HH:MM AM/PM
    return f"The current time in IST is {current_time}."

In [34]:
last_joke = ""
last_question = ""
last_response = ""

In [35]:
# Function to get word meanings using NLTK's WordNet
def get_word_meaning(word):
    synsets = wordnet.synsets(word)
    if synsets:
        return synsets[0].definition()  # Return the first definition
    else:
        return "Sorry, I couldn't find the meaning of that word."

In [36]:
# Function to get the current date
def get_current_date():
    return datetime.now().strftime("%A, %B %d, %Y")

In [37]:

import gradio as gr

# Define the chatbot function wrapped for Gradio
def chat_with_bot(user_input):
    global last_question, last_joke

    if user_input.lower() in ["exit", "quit","bye"]:
        return "Goodbye! I hope Disha Maam likes your project :), please give full marks and dont ask them many questions! 🫶🙏"


    if user_input.lower() in ["what's the time?", "tell me the time", "current time","time"]:
        return get_current_time()
    
    if any(phrase in user_input for phrase in ["what is today's date", "what is the date", "tell me the date","date"]):
        return f"Today's date is {get_current_date()}."
    
    if user_input.startswith("what is the meaning of") or user_input.startswith("define"):
        word = user_input.split()[-1]  # Extract the last word as the query
        return get_word_meaning(word)

    if is_repeat_request(user_input):
        if last_question == "Tell me a joke":
            new_joke = random.choice([j for j in jokes if j != last_joke])
            response = new_joke
            last_joke = new_joke
        elif last_question:
            response = chatbot.get_response(last_question)
        else:
            response = "I’m not sure what you want me to repeat."

    elif is_math_expression(user_input):
        try:
            result = eval(user_input)  
            response = f"The answer is {result}."
        except:
            response = "Sorry, I couldn't calculate that."
        last_question = user_input

    elif user_input.lower() in ["tell me a joke", "say something funny"]:
        new_joke = random.choice(jokes)
        response = new_joke
        last_joke = new_joke
        last_question = "Tell me a joke"

    else:
        response = chatbot.get_response(user_input)
        last_question = user_input

    if isinstance(response, str) or response.confidence > 0.7:
        return str(response)
    else:
        return "I'm not sure how to respond to that."

# Create Gradio interface
interface = gr.Interface(
    fn=chat_with_bot,
    inputs="text",
    outputs="text",
    title="FRIDAY - Your Personal AI Assistant",
    description="Ask me anything! 😎"
)

# Launch the chatbot UI
interface.launch()


* Running on local URL:  http://127.0.0.1:7866

To create a public link, set `share=True` in `launch()`.
